In [4]:
import sys
sys.path.append('..')
print(sys.path)
from DeepSolarEye.dl_logic.model import regression_ResNet

import tensorflow as tf
import pandas as pd
import numpy as np
import os
import cv2
from datetime import datetime

from tensorflow.keras.callbacks import EarlyStopping

['/home/hanswerners/code/HWWer/Deep_Solar_Eye/notebooks', '/home/hanswerners/code/HWWer/04-Decision-Science/01-Project-Setup/data-context-and-setup', '/home/hanswerners/code/HWWer/Deep_Solar_Eye/notebooks', '/home/hanswerners/.pyenv/versions/3.10.6/lib/python310.zip', '/home/hanswerners/.pyenv/versions/3.10.6/lib/python3.10', '/home/hanswerners/.pyenv/versions/3.10.6/lib/python3.10/lib-dynload', '', '/home/hanswerners/.pyenv/versions/3.10.6/envs/Deep_Solar_Eye/lib/python3.10/site-packages', '..', '..']


In [6]:
def preprocess_data(size=('full', 'noon', '15_mins')) -> (pd.DataFrame, np.ndarray):
    """
    Preprocesses images from file. Returns metadata in a dataframe, and a np array of image data.
    Use 'size' kwarg to decide what split of the dataset will be processed and returned.
    'full' = c. 45k images
    'noon' = c. 3.7k images
    '15_mins' = c. 1k images

    Returns: Metadata Dataframe, Tensor np.ndarray
    """

    folder_path = "../raw_data/PanelImages"
    image_data = [] # initialise an empty array to stack the images
    metadata = []
    # Regular expression pattern to extract date and intensity values from the filename
    # Regular expression pattern to extract date and intensity values from the filename
    minute_range = np.arange(0, 15, 1)
    # Convert the numpy array to a list of strings
    minute_range_strings = [str(num) for num in minute_range]
    read_count = 0

    # capped at 1000 for now
    for filename in os.listdir(folder_path):
        if not filename.endswith(".jpg"):
            continue
        split_name = filename.split('_')
        hour = split_name[4]
        minute = split_name[6]
        # put in the break
        if size in ['noon', '15_mins'] and hour != '12':
            continue
        if size == '15_mins' and minute not in minute_range_strings:
            continue
        read_count += 1
        weekday = split_name[1]
        month = split_name[2]
        day = split_name[3]
        second = split_name[8]
        year = split_name[9]
        datetime_obj = datetime.strptime(f"{month} {day} {year} {hour}:{minute}:{second}", "%b %d %Y %H:%M:%S")
        age_loss = split_name[11]
        irradiance_level = split_name[13][:-4]

        # append metadata to list
        filename_info = [month, weekday, day, hour, minute, second, year, datetime_obj, age_loss, irradiance_level]

        metadata.append(filename_info)
    df = pd.DataFrame(metadata, columns=['Month', 'Day', 'Date', 'Hour', 'Minute', 'Second', 'Year',
                                        'Datetime', 'Percentage Loss', 'Irradiance Level'])

    df = df.astype({'Month': str, 'Day': str, 'Date': int, 'Hour': int, 'Minute': int, 'Second': int, 'Year': int,

                                       'Datetime': 'datetime64[ns]', 'Percentage Loss': float, 'Irradiance Level': float})
    return df

In [19]:
df = preprocess_data('full')
x_df = df[['Hour', 'Irradiance Level']]
y_df = df[['Percentage Loss']]

In [32]:
path_imgs = ('../raw_data/PanelImages/*.jpg')
images = tf.data.Dataset.list_files(path_imgs, shuffle=False)

def load_and_process_image(path_imgs):
    img = tf.io.read_file(path_imgs)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255
    return img


#def load_images(path_imgs):
#  return tf.image.decode_image(tf.io.read_file(path_imgs))

#images_ds = images.map(load_images).batch(16)
images_ds = images.map(load_and_process_image).batch(16)

In [15]:
df_ds = tf.data.Dataset.from_tensor_slices(df[['Hour', 'Irradiance Level']])
y_ds= tf.data.Dataset.from_tensor_slices(df[['Percentage Loss']])

x_ds = tf.data.Dataset.zip((images_ds, df_ds))
all_ds = tf.data.Dataset.zip((x_ds, y_ds))

df_ds = tf.data.Dataset.from_tensor_slices(df[['Hour', 'Irradiance Level']].values.astype(np.float32))
df_ds = df_ds.batch(16)
y_ds = tf.data.Dataset.from_tensor_slices(df[['Percentage Loss']].values.astype(np.float32))
y_ds = y_ds.batch(16)

x_ds = tf.data.Dataset.zip((images_ds, df_ds))
all_ds = tf.data.Dataset.zip((x_ds, y_ds))

In [5]:
model = regression_ResNet(model_name='ResNet50', input_shape=(224, 224, 3),input_time_irradiance=(2,), num_units=512, pretrained=True)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])


2024-02-08 11:51:17.301241: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/hanswerners/.pyenv/versions/3.10.6/envs/Deep_Solar_Eye/lib/python3.10/site-packages/cv2/../../lib64:
2024-02-08 11:51:17.302401: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-02-08 11:51:17.302605: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Hans): /proc/driver/nvidia/version does not exist
2024-02-08 11:51:17.303195: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler 

In [16]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True
)

history = model.fit(all_ds,
            epochs=2,
            batch_size=16,
            callbacks=early_stopping
           )

Epoch 1/2
   3/2860 [..............................] - ETA: 3:11:52 - loss: 23.3870 - mae: 3.7684

KeyboardInterrupt: 

In [2]:
import sys
sys.path.append('..')
print(sys.path)
from DeepSolarEye.handling.load_tensor import get_numerical_data, load_and_process_image, load_tensor
%load_ext autoreload
%autoreload 2

['/home/hanswerners/code/HWWer/Deep_Solar_Eye/notebooks', '/home/hanswerners/code/HWWer/04-Decision-Science/01-Project-Setup/data-context-and-setup', '/home/hanswerners/code/HWWer/Deep_Solar_Eye/notebooks', '/home/hanswerners/.pyenv/versions/3.10.6/lib/python310.zip', '/home/hanswerners/.pyenv/versions/3.10.6/lib/python3.10', '/home/hanswerners/.pyenv/versions/3.10.6/lib/python3.10/lib-dynload', '', '/home/hanswerners/.pyenv/versions/3.10.6/envs/Deep_Solar_Eye/lib/python3.10/site-packages', '..']


2024-02-08 11:50:44.991552: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-08 11:50:50.288647: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-02-08 11:50:50.288829: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-02-08 11:50:55.210798: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-08 11:51:05.018611: W tensorflow/stream_executor/platform/de

In [3]:
df = get_numerical_data()


In [4]:
df

,Seconds of Day,Percentage Loss,Irradiance Level
0,54074,0.553858,0.326784
1,61531,0.138957,0.214173
2,59298,0.014388,0.095125
3,39837,0.882971,0.294816
4,50073,0.104878,0.667071
...,...,...,...
45749,35553,0.143640,0.544498
45750,24558,0.584764,0.052094
45751,59307,0.026915,0.282227
45752,56170,0.140307,0.513020


In [6]:
all_ds = load_tensor(df, 16)

In [8]:
type(all_ds)

tensorflow.python.data.ops.dataset_ops.ZipDataset

In [7]:
dataset_size = sum(1 for _ in all_ds)

In [10]:
dataset_size

2860

In [8]:
train_size = int(0.6 * dataset_size)
test_size = int(0.2 * dataset_size)
validation_size = int(0.2 * dataset_size)


In [12]:
#all_ds = all_ds.shuffle(buffer_size=dataset_size)
#"../raw_data/PanelImages/*.jpg"

In [9]:
train_ds = all_ds.take(train_size)
test_ds = all_ds.skip(train_size).take(test_size)
validation_ds = all_ds.skip(train_size + test_size)

In [10]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True
)

history = model.fit(train_ds,
            epochs=2,
            batch_size=16,
            callbacks=early_stopping,
            validation_data=validation_ds
           )

Epoch 1/2
   9/1716 [..............................] - ETA: 1:07:02 - loss: 159445.8906 - mae: 337.5943

KeyboardInterrupt: 